In [1]:
import pandas as pd
import numpy as np

In [44]:
df_bow = pd.read_csv(r'C:\Local Disk F\ML\Campus SuperBowl Challenge 2024\AMEX_2024\bowler_level_scorecard.csv')

In [45]:
df_bow.head()

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.50,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.50,0,7.0,3,1,2,0,2021-01-01
2,8638034,JS Nm,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,23,2,24,5.75,0,12.0,1,1,0,0,2021-01-01
3,8638034,LV vn Bk,3083667.0,NED:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,31,1,24,7.75,0,9.0,4,0,2,0,2021-01-01
4,8638034,PF Yd,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,1,21,1,24,5.25,0,8.0,0,0,1,0,2021-01-01


In [46]:
df_bow[['bowler_country', 'bowler_bat', 'bowler_bowl', 'extra']] = df_bow['bowler_details'].str.split(':', expand=True)

# Drop the original columns
df_bow.drop(columns=['bowler_details', 'extra'], inplace=True)

In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 10))
# scaler2 = MinMaxScaler(feature_range=(0, 0.1))

In [48]:
# Aggregate the data
bowler_stats = df_bow.groupby('bowler_id').agg({
    'runs': 'sum',
    'wicket_count': 'sum',
    'balls_bowled': 'sum',
    'maiden': 'sum',
    'dots': 'sum',
    'Fours': 'sum',
    'Sixes': 'sum',
    'wides': 'sum',
    'noballs': 'sum',
    'match id': 'count'  # Assuming each row represents one match
}).reset_index()

# Rename columns for clarity
bowler_stats.rename(columns={'match id': 'matches'}, inplace=True)

# Calculate additional metrics
bowler_stats['economy_rate'] = (bowler_stats['runs'] / bowler_stats['balls_bowled']) * 6
bowler_stats['economy_rate'] = scaler.fit_transform(bowler_stats[['economy_rate']])

bowler_stats['average_wickets'] = bowler_stats['wicket_count'] / bowler_stats['matches']
bowler_stats['average_wickets'] = scaler.fit_transform(bowler_stats[['average_wickets']])

bowler_stats['dots_per_over'] = (bowler_stats['dots'] / bowler_stats['balls_bowled']) * 6
bowler_stats['dots_per_over'] = scaler.fit_transform(bowler_stats[['dots_per_over']])

bowler_stats['noballs_per_over'] = (bowler_stats['noballs'] / bowler_stats['balls_bowled']) * 6
bowler_stats['noballs_per_over'] = scaler.fit_transform(bowler_stats[['noballs_per_over']])

bowler_stats['fours_per_over'] = (bowler_stats['Fours'] / bowler_stats['balls_bowled']) * 6
bowler_stats['fours_per_over'] = scaler.fit_transform(bowler_stats[['fours_per_over']])

bowler_stats['sixes_per_over'] = (bowler_stats['Sixes'] / bowler_stats['balls_bowled']) * 6
bowler_stats['sixes_per_over'] = scaler.fit_transform(bowler_stats[['sixes_per_over']])

# Define the performance score (example weights)
bowler_stats['bowl_performance_score'] = (
    bowler_stats['average_wickets'] * 0.3 -
    (bowler_stats['economy_rate']) * 0.3 +
    bowler_stats['dots_per_over'] * 0.15
    # bowler_stats['maiden'] * 0.1 -
     - bowler_stats['noballs_per_over'] * 0.1
     - bowler_stats['sixes_per_over'] * 0.1
     - bowler_stats['fours_per_over'] * 0.05
)

# Sort by performance score to see the top performers
bowler_stats = bowler_stats.sort_values(by='bowl_performance_score', ascending=False)

# Display the result
print(bowler_stats)

      bowler_id  runs  wicket_count  balls_bowled  maiden  dots  Fours  Sixes  \
1159  8059015.0    20             4            24       0  15.0      3      0   
1536  9822392.0     7             3            18       0  14.0      1      0   
532   3896801.0    27             4            24       0  14.0      4      1   
1020  7534722.0    28             4            24       0  13.0      2      2   
1511  9382967.0    20             4            12       0   6.0      4      0   
...         ...   ...           ...           ...     ...   ...    ...    ...   
1132  7960966.0    23             0             6       0   0.0      4      1   
20     151052.0    43             0            12       0   1.0      1      5   
594   4171495.0     4             0             1       0   0.0      1      0   
806   5652296.0    20             0             6       0   0.0      0      2   
411   3248783.0    18             0             4       0   1.0      0      3   

      wides  noballs  match

In [49]:
bowler_stats['bowl_performance_score'] = scaler.fit_transform(bowler_stats[['bowl_performance_score']])

In [50]:
bowler_stats.head()

,bowler_id,runs,wicket_count,balls_bowled,maiden,dots,Fours,Sixes,wides,noballs,matches,economy_rate,average_wickets,dots_per_over,noballs_per_over,fours_per_over,sixes_per_over,bowl_performance_score
1159,8059015.0,20,4,24,0,15.0,3,0,0,1,1,1.851852,10.0,6.250000,1.5,1.250000,0.000000,10.000000
1536,9822392.0,7,3,18,0,14.0,1,0,0,0,1,0.864198,7.5,7.777778,0.0,0.555556,0.000000,9.941401
532,3896801.0,27,4,24,0,14.0,4,1,0,0,1,2.500000,10.0,5.833333,0.0,1.666667,0.555556,9.730172
1020,7534722.0,28,4,24,0,13.0,2,2,0,0,1,2.592593,10.0,5.416667,0.0,0.833333,1.111111,9.576860
1511,9382967.0,20,4,12,0,6.0,4,0,2,0,1,3.703704,10.0,5.000000,0.0,3.333333,0.000000,8.973835


In [51]:
bowler_stats.to_csv(r'C:\Local Disk F\ML\Campus SuperBowl Challenge 2024\Data\bowler_stats.csv', index = False)